In [1]:
from nanomesh.generator import Generator
import SimpleITK as sitk
from nanomesh.utils import show_slice, show_volume, generate_mesh_from_binary_image
import numpy as np
import math
import pygalmesh

#### Create the ideal generator

The actual dimension of the cells are 480nm x 680nm x 480 nm However we specify xdim=ydim=zdim because the perdiodic mesher can only handle cubic domains


In [2]:
# dimension of the domain used by CGAL to mesh the data
xdim = ydim = zdim = 0.48

class Pore3D(pygalmesh.DomainBase):
    def __init__(self):
        super().__init__()
        
        self.gen = Generator(680, 480, 0.24*680)

        # Possible rotation/transformation of the coordinate system
        theta = 0.5 * math.pi 
        c = math.cos(theta)
        s = math.sin(theta)
        trans = np.array([
            [ c, 0, s],
            [ 0, 1, 0],
            [-s, 0, c]
        ])
        
        # number of voxel in each direction
        self.size = [48,68,48]
        
        # size of the voxel (in nm)
        self.res = [10,10,10]
        
        # generate the datacube
        self.data = self.gen.generate_vect(self.size, 
                                           self.res, 
                                           transform=trans, 
                                           bin_val=[0.,1.])

                
        # size used during the meshing by the orcale
        # must be a perfect cube
        self.x = np.linspace(0,xdim,self.data.shape[0])
        self.y = np.linspace(0,ydim,self.data.shape[1])
        self.z = np.linspace(0,zdim,self.data.shape[2])
        
        
    def eval(self, x):
        """Orale function used during the meshing."""

        # get the indices
        ix = np.digitize(x[0] % xdim,self.x)-1
        iy = np.digitize(x[1] % ydim,self.y)-1
        iz = np.digitize(x[2] % zdim,self.z)-1
        
        # evaluate the data cube
        if self.data[ix,iy,iz] == 0:
            out = -1.
        else:
            out = 1.
        return out
    

In [3]:
pore = Pore3D()
im = sitk.GetImageFromArray(pore.data.astype('uint8'))
show_slice(im, dim='z',scale=2)

interactive(children=(IntSlider(value=23, description='z', max=47), Output()), _dom_classes=('widget-interact'…

### Instantiate the pore and visualize a section of the dat

In [5]:
class FullCube(pygalmesh.DomainBase):
    
    def __init__(self):
        super().__init__()

    def eval(self,x):
        return -1

In [9]:
mesh = pygalmesh.generate_periodic_mesh_multiple_domains(
    [Pore3D(),FullCube()],
    ["--","+-"],
    [0,0,0,xdim,ydim,zdim],
    max_cell_circumradius=0.025,
    min_facet_angle=30,
    max_radius_surface_delaunay_ball=0.025,
    max_facet_distance=0.025,
    max_circumradius_edge_ratio=2.0,
    number_of_copies_in_output=1,
    exude=False,
    perturb=False,
    odt=False,
    lloyd=False,
    verbose=True
    )

In [8]:
import pyvista as pv
pv.plot_itk(mesh)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [7]:
from nanomesh.periodic_utils import insert_periodic_info
mesh = insert_periodic_info(mesh, [0,0,0,1.,0.68,1.])

In [8]:
mesh.write('nanopt_full_period_2domain_all_connections.gmsh', file_format="gmsh22", binary=False)